In [5]:
import pandas as pd
pd.set_option('display.max_rows', 500)
import plotly.express as px
from pathlib import Path

# Point 4: Accelerating the Shift to Zero Emission Vehicles, 
End the sale of new pure 
petrol and diesel cars and 
vans by 2030 and consult on 
phase out for diesel HGVs

## What % of cars on the road today are Zero Emission?

In [6]:
make_model_df = pd.read_csv(Path("raw_data", "vehicles", "by_make_model.csv"))
make_model_df = make_model_df[make_model_df['LicenceStatus'] == 'Licensed']

quarters_cols = ['2022Q3',
       '2022Q2', '2022Q1', '2021Q4', '2021Q3', '2021Q2', '2021Q1', '2020Q4',
       '2020Q3', '2020Q2', '2020Q1', '2019Q4', '2019Q3', '2019Q2', '2019Q1',
       '2018Q4', '2018Q3', '2018Q2', '2018Q1', '2017Q4', '2017Q3', '2017Q2',
       '2017Q1', '2016Q4', '2016Q3', '2016Q2', '2016Q1', '2015Q4', '2015Q3',
       '2015Q2', '2015Q1', '2014Q4', '2014Q3', '2014Q2', '2014Q1', '2013Q4',
       '2013Q3', '2013Q2', '2013Q1', '2012Q4', '2012Q3', '2012Q2', '2012Q1',
       '2011Q4', '2011Q3', '2011Q2', '2011Q1', '2010Q4', '2010Q3', '2010Q2',
       '2010Q1', '2009Q4', '2009Q3', '2009Q2', '2009Q1', '2008Q4', '2008Q3',
       '2007Q4', '2006Q4', '2005Q4', '2004Q4', '2003Q4', '2002Q4', '2001Q4',
       '2000Q4', '1999Q4', '1998Q4', '1997Q4', '1996Q4', '1995Q4', '1994Q4']

# make_model_df['total'] = make_model_df.sum(numeric_only = True, axis = 1)

make_model_df = make_model_df[quarters_cols].T
make_model_df['total'] = make_model_df.sum(numeric_only = True, axis = 1)
make_model_df = make_model_df.iloc[::-1] #Reversing index for chronological order
make_model_df
fig = px.line(make_model_df, x=make_model_df.index, y="total", title='Total vehicles licensed in the UK', template='plotly_dark', 
              labels={
                     "index": "Date",
                     "total": "Num. Vehicles",
                 },)
fig.show()

In [7]:
make_model_df = make_model_df.reset_index()
make_model_df.head()
make_model_df = make_model_df[['index', 'total']]
make_model_df['date_stripped' ] = make_model_df['index'].str.slice(0,4) + " " + make_model_df['index'].str.slice(4, 6)
make_model_df= make_model_df.rename(columns = {'total':'total licensed vehicles'})
make_model_df = make_model_df.drop('index', axis = 1)

In [8]:
piv_df_raw = pd.read_excel(Path("raw_data","vehicles","veh0141_PIV.ods"), engine="odf", sheet_name='VEH0141a_Fuel', skiprows = 4)
piv_df = piv_df_raw.copy()

In [9]:
piv_df = piv_df[piv_df['Geography [note 2]'] == 'United Kingdom']
piv_df = piv_df[piv_df['Units'] == 'Number']
piv_df = piv_df.drop(['Geography [note 2]', 'Units', 'Battery Electric','Plug-in hybrid electric (petrol)',	'Plug-in hybrid electric (diesel)',	'Range extended electric'], axis = 1)

In [10]:
def strip_date(string:str):
    years = []
    for i, _ in enumerate(range(150)):
        year = 1900 + i
        years.append(str(year)) 
    quarters = ["Q1", "Q2", "Q3", "Q4"]
    to_keep = years + quarters
    return ' '.join([x for x in string.split() if x in to_keep])

In [11]:
piv_df['date_stripped'] = piv_df['Date'].apply(lambda x: strip_date(x))
piv_df['Total'] = piv_df['Total'].astype(float)
piv_df_grouped = piv_df.groupby('date_stripped').sum(numeric_only = True)

In [12]:
fig = px.line(piv_df_grouped, x=piv_df_grouped.index, y="Total", title='Plug in vehicles licensed in the UK', template='plotly_dark', 
                            labels={
                     "date_stripped": "Date",
                     "Total": "Num. Vehicles",
                 }) # This is all vehicles - HGV, Buses, motorbikes, cars
fig.show()

In [13]:
piv_df_grouped = piv_df_grouped.rename(columns = {'Total':'total plug in vehicles licensed'}).reset_index()

In [14]:
merged_df = make_model_df.merge(piv_df_grouped, how = 'left')

In [25]:
fig = px.line(merged_df, x='date_stripped',
              y=["total plug in vehicles licensed","total licensed vehicles"], 
              title='Plug in vehicles licensed in the UK', 
              template='plotly_dark', 
               labels={
                     "date_stripped": "Date",
                     "value": "Num. Vehicles",
                 }) # This is all vehicles - HGV, Buses, motorbikes, cars
fig.update_layout(legend_title_text=' ')
fig.update_layout(legend=dict(
    yanchor="top",
    y=1.4,
    xanchor="left",
    x=0.4
))
fig.show()

Back to the original question - what % of cars today are zero emission? Note the only data available is for plug in cars, not necessarily zero emission (includes hybrids)

In [16]:
merged_df['% Plug in vehicles'] = 100*(merged_df['total plug in vehicles licensed'] / merged_df['total licensed vehicles'])

In [26]:
merged_df.tail(1)

,total licensed vehicles,date_stripped,total plug in vehicles licensed,% Plug in vehicles
70,39519623,2022 Q3,2006144.0,5.076324


# How many new cars are added to the roads every year?

In [29]:
merged_df['vehicles added'] = merged_df['total licensed vehicles'].diff()

In [30]:
merged_df.head()

,total licensed vehicles,date_stripped,total plug in vehicles licensed,% Plug in vehicles,vehicles added
0,25231451,1994 Q4,NaN,NaN,NaN
1,25369353,1995 Q4,NaN,NaN,137902.0
2,26301921,1996 Q4,NaN,NaN,932568.0
3,26973790,1997 Q4,NaN,NaN,671869.0
4,27538415,1998 Q4,NaN,NaN,564625.0


In [33]:
fig = px.line(merged_df, x='date_stripped', 
              y="vehicles added", 
              title='Total vehicles added each year', 
              
              template='plotly_dark', 
              labels={
                     "date_stripped": "Date",
                     "Total": "Num. Vehicles",

                 }) # This is all vehicles - HGV, Buses, motorbikes, cars
fig.show()

In [36]:
merged_df['piv vehicles added'] = merged_df['total plug in vehicles licensed'].diff()

In [38]:
fig = px.line(merged_df, x='date_stripped', 
              y="piv vehicles added", 
              title='Plug in vehicles added each year', 
              
              template='plotly_dark', 
              labels={
                     "date_stripped": "Date",
                     "Total": "Num. Vehicles",

                 }) # This is all vehicles - HGV, Buses, motorbikes, cars
fig.show()

In [40]:
fig = px.line(merged_df, x='date_stripped',
              y=["piv vehicles added","vehicles added"], 
              title='Vehicles added each year', 
              template='plotly_dark', 
               labels={
                     "date_stripped": "Date",
                     "value": "Num. Vehicles",
                 }) # This is all vehicles - HGV, Buses, motorbikes, cars
fig.update_layout(legend_title_text=' ')
fig.update_layout(legend=dict(
    yanchor="top",
    y=1.4,
    xanchor="left",
    x=0.4
))
fig.show()

## Whats the average cost of a zero emission car compared to petrol/diesel?